In [1]:
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

# Contents
1. Basic autograd example 1
2. Basic autograd example 2
3. Loading data from numpy
4. Implementing the input pipline
5. Input pipline for custom dataset
6. Using pretrained model
7. Save and load model

## 1. Basic autograd example 1 

In [2]:
# Create tensor
x = Variable(torch.Tensor([1]), requires_grad=True)
w = Variable(torch.Tensor([2]), requires_grad=True)
b = Variable(torch.Tensor([3]), requires_grad=True)

In [3]:
print(x, w, b)

Variable containing:
 1
[torch.FloatTensor of size 1]
 Variable containing:
 2
[torch.FloatTensor of size 1]
 Variable containing:
 3
[torch.FloatTensor of size 1]



In [4]:
# Build a computational graph.
y = w * x + b # y = 2* x + 3

In [5]:
# Compute gradients.
y.backward()

In [6]:
# Print out the gradients.
print(x.grad) # x.grad = 2
print(w.grad) # w.grad = 1
print(b.grad) # b.grad = 1

Variable containing:
 2
[torch.FloatTensor of size 1]

Variable containing:
 1
[torch.FloatTensor of size 1]

Variable containing:
 1
[torch.FloatTensor of size 1]



## 2. Basic autograd example 2

In [7]:
# Create tensors.
x = Variable(torch.randn(5, 3))
y = Variable(torch.randn(5, 2))

In [8]:
# Build a linear layer.
linear = nn.Linear(3, 2)
print('w: ', linear.weight)
print('b: ', linear.bias)

w:  Parameter containing:
 0.4006 -0.1869  0.3570
-0.2671 -0.2712 -0.0969
[torch.FloatTensor of size 2x3]

b:  Parameter containing:
-0.5311
-0.5534
[torch.FloatTensor of size 2]



In [9]:
# Build Loss and Optimizer.
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

In [10]:
# Forward propagation
pred = linear(x)

In [11]:
# Computation loss.
loss = criterion(pred, y)
print('loss: ', loss.data[0])

loss:  0.46748727560043335


In [12]:
# Backpropagation.
loss.backward()

In [13]:
# Printout the gradients.
print('dL/dw: ', linear.weight.grad)
print('dl/db: ', linear.bias.grad)

dL/dw:  Variable containing:
 0.3775  0.0541  0.0682
-0.4223  0.1445 -0.1875
[torch.FloatTensor of size 2x3]

dl/db:  Variable containing:
 0.1923
-0.1014
[torch.FloatTensor of size 2]



In [14]:
# 1-step Optimization (gradient descent)
optimizer.step()

In [15]:
# You can also do optimization at the low level as shown below
# linear.weight.data.sub_(0.01 * linear.weight.grad.data)
# linear.bias.data_sub_(0.01 * linear.bias.grad.data)

In [16]:
# Print out the loss after optimization.
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.data[0])

loss after 1 step optimization:  0.46319660544395447


## 3. Loading data from numpy

In [17]:
a = np.array([[1, 2], [3, 4]])
b = torch.from_numpy(a) # convert numpy array to torch tensor
c = b.numpy() # convert torch tensor to numpy array

In [18]:
print(a)
print(b)
print(c)

[[1 2]
 [3 4]]

 1  2
 3  4
[torch.IntTensor of size 2x2]

[[1 2]
 [3 4]]


## 4. Implementing the input pipline

In [19]:
# Download and construct dataset.
train_dataset = dsets.CIFAR10(root='./data/',
                              train=True,
                              transform=transforms.ToTensor(),
                              download=True)

Files already downloaded and verified


In [20]:
# Select one data pair(read data from disk)
image, label = train_dataset[0]
print(image.size())
print(label)

torch.Size([3, 32, 32])
6


In [21]:
# Data Loader(this provides queue and thread in a very simple way)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100,
                                           shuffle=True,
                                           num_workers=2)

In [22]:
# When iteration starts, queue and thread start to load dataset from files.
data_iter = iter(train_loader)

In [23]:
# mini-batch images and labels.
images, labels = data_iter.next()
print(images.size())
print(labels.size())

torch.Size([100, 3, 32, 32])
torch.Size([100])


In [24]:
# Acutal usage of data loader is as below.
for images, labels in train_loader:
    # Your training code will be written here
    pass

## 5. Input pipline for custom dataset

In [25]:
# You should build custom dataset as below
class CustomDataset(data.Dataset):
    def __init__(self):
        # TODO
        # 1. initialize file path or list of file names.
        pass
    def __getitem__(self, index):
        # TODO
        # 1. Read one data from file (e.g using numpy.fromfile, PIL.Image.open).
        # 2. Preprocess the data (e.g torchvision.Transform).
        # 3. Return a data pair (e.g image and label).
        pass
    def __len__(self):
        # You should change 0 to the total size of your dataset.
        return 0
    
# Then, you can just use prebuilt torch`s data loader.
custom_dataset = CustomDataset()
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                           batch_size=100,
                                           shuffle=True,
                                           num_workers=2)

## 6. Using pretrained model

In [65]:
# Download and load pretrained resnet.
resnet = torchvision.models.resnet18(pretrained=True)

In [66]:
# If you want to finetune only top layer of the model.
for param in resnet.parameters():
    param.requires_grad=False

In [71]:
# Replace top layer for finetuning.
resnet.fc = nn.Linear(resnet.fc.in_features, 100)

In [72]:
# For test.
images = Variable(torch.randn(10, 3, 224, 224))
outputs = resnet(images)
print(outputs.size())

torch.Size([10, 100])


## 7. Save and load the model

In [73]:
# Save and load the entire model.
torch.save(resnet, 'model.pkl')
model = torch.load('model.pkl')

In [75]:
# Save and load the only the model parameters(recommended)
torch.save(resnet.state_dict(), 'params.pkl')
resnet.load_state_dict(torch.load('params.pkl'))